# Compare Designs by altering Spring K and Damping C

## Save Figures?

In [ ]:
SAVE_FIG = False

## Imports

In [ ]:
import csv
import re
import glob
import datetime
from pathlib import Path
import pandas
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from pogo_stick_jumping.jumping_ODE_Nonlinear import PogoODEnonlinear

## Set Up the Save Paths

In [ ]:
current_path = Path.cwd()
save_path = current_path / "Compare_Data"
if not os.path.exists(save_path): os.makedirs(save_path)

## Set up env and run tests with different values of K and C

In [ ]:
m_rod = 0.175               # mass of the pogo-stick rod (kg)
m_act = 1.003               # mass of the pogo-stick rod (kg)
mass = m_rod + m_act        # total mass (kg)
f = 11.13                   # natural freq. (rad)
wn = f * (2 * np.pi)        # Robot frequency (rad/s)
zeta = 0.01                 # Robot damping ratio
c = 2 * zeta * wn * mass    # Calculate damping coeff
k = mass * wn**2            # Calulate spring constant
A_max = 63.2      # max acceleration of actuator (m/s^2)
V_max = 2.0       # max velocity of actuator (m/s)
Distance = 0.008  # Distance to move actuator in jump command (m)
Spacing = 0.75 * (1 / f)  # Space commands by 0.5*period of oscillation


x_init = 0.0
x_dot_init = 0.0
x_act_init = 0.0
x_act_dot_init = 0.0

spring_k = 5760
zeta = 0.01
variance = 0.75
min_spring_k = spring_k - variance * spring_k
max_spring_k = spring_k + variance * spring_k
min_zeta = zeta - variance * zeta
max_zeta = zeta + variance * zeta

spring_ks = np.linspace(min_spring_k, max_spring_k, 3, endpoint=True)
zetas = np.linspace(min_zeta, max_zeta, 3, endpoint=True)

for k in spring_ks:
    for zeta in zetas:
        
        pogo_stick = PogoODEnonlinear(m_act, m_rod, k, zeta, A_max, V_max, Distance, Spacing)

        x0 = [x_init, x_dot_init, x_act_init, x_act_dot_init]
        sim_time, timeseries = pogo_stick.run_simulation(x0, duration=1.0)
        data = timeseries[0]
        header = "x"
        data_name = f"kp{k:.4f}kv{zeta:.4f}"
        data_path = save_path / f"{data_name}.csv"
        np.savetxt(data_path, data, header=header, delimiter=',')


## Load in the Results

In [ ]:
# sort the files read in
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

files = sorted(glob.glob(str(save_path / '*.csv')), key=numericalSort)

print(f'Number of Agents found: {len(files)}\n')

# organize the data into dictionaries
data = {}                                                                                   # dict to store agent data
file_names = []                                                                             # list to store file names for naming agents in dict
for f in files:
    file_name = os.path.basename(f)                                                         # get file name and split it up by
    file_name = re.split('[_ .]', file_name)
    # uncomment print statment to determine what numbers are needed in file_name
    print(file_name)
    file_name = f"{file_name[0]}-{file_name[1]}{file_name[2]}"                                                 # choose the parts of file name you want to keep using the previous print statment
    file_names.append(file_name)                                                            # add the agents name to the list of file names
    df = pandas.read_csv(f)                                                                 # read in the first .csv file
    df_columns = list(df)                                                                   # get the list of data column names
    for column in df_columns:                                                               
        df_columns[df_columns.index(column)] = column.replace(' ', '').replace('#', '')     # get rid of spaces and #'s in the column names
    df.columns = df_columns                                                                 # set the .csv file column names to the cleaned up names
    data[file_name] = df.to_dict('series')                                                  # turn the .csv into a dict and place it in the data dict

    for column in data[file_name]:
            data[file_name][column] = data[file_name][column].to_numpy()                    # convert the columns in the agent dict in the data dict to numpy arrays
print(f'\nAgent Key Names: {list(data)}\n')                                                    # print out the agents names for using later in plotting
print(f"""\nAgent's Data Key Names: {list(data[next(iter(data))])}\n""")                       # print out the data column names for using later in plotting                                                # print out the agents names for using later in plotting

## Plot the Results

In [ ]:
# Set the plot size - 3x2 aspect ratio is best
fig = plt.figure(figsize=(12,12))
ax = plt.gca()

# Define the X and Y axis labels
plt.xlabel(r'Time (s)', fontsize=22, weight='bold', labelpad=5)
plt.ylabel(r'Height (m)', fontsize=22, weight='bold', labelpad=10)

for design in data:
    height = data[design]["x"]
    time = np.arange(len(height))
    plt.plot(time, height, linewidth=2, linestyle='-', label=design)

# uncomment below and set limits if needed
# plt.xlim(0,4)
# plt.ylim(bottom=None, top=.35)

# Create the legend, then fix the fontsize
leg = plt.legend(loc='upper right', ncol = 1, fancybox=True)
ltext  = leg.get_texts()
plt.setp(ltext,fontsize=18)

# Adjust the page layout filling the page using the new tight_layout command
plt.tight_layout(pad=0.5)

# save the figure as a high-res pdf in the current folder
filename = 'RewardVsTime_{}.png'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S'))
path = save_path / filename
if SAVE_FIG is True:
    plt.savefig(path)
plt.show()

In [8]:
minimum = 0
maximum = 0
for design in data:
    if min(data[design]['x']) < minimum:
        minimum = min(data[design]['x'])
    if max(data[design]['x']) > maximum:
        maximum = max(data[design]['x'])
print(f"Max Height is: {maximum}")
print(f"Minimum Height is: {minimum}")

Max Height is: 0.13089527713757743
Minimum Height is: -0.010980637207858152
